In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
import time
from pyspark.sql import SparkSession
from pyspark import SparkFiles
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Suppress logging messages for SparkContext
spark.sparkContext.setLogLevel("ERROR")

import warnings
# Filter hostname resolution warning
warnings.filterwarnings("ignore", category=UserWarning, module="py4j.*")

23/12/05 21:10:39 WARN Utils: Your hostname, Terrys-iMac.local resolves to a loopback address: 127.0.0.1; using 192.168.1.27 instead (on interface en1)
23/12/05 21:10:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/05 21:10:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [6]:
# 2b. Check out the data types of the DataFrame. 
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("""
    SELECT bedrooms, YEAR(CAST(date AS DATE)) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = '4' 
    GROUP BY bedrooms, YEAR(CAST(date AS DATE))
    ORDER BY bedrooms, YEAR(CAST(date AS DATE)) 
""")

# Show the result
result.show()

+--------+----+---------+
|bedrooms|year|avg_price|
+--------+----+---------+
|       4|2019| 300263.7|
|       4|2020|298353.78|
|       4|2021|301819.44|
|       4|2022|296363.88|
+--------+----+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("""
    SELECT date_built, ROUND(AVG(CAST(price AS DOUBLE)), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = '3' AND bathrooms = '3' 
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [78]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = spark.sql("""
    SELECT date_built, ROUND(AVG(CAST(price AS DOUBLE)), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = '3'
      AND bathrooms = '3'
      AND floors = '2' 
      AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [79]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
result = spark.sql("""
    SELECT view, ROUND(AVG(CAST(price AS DOUBLE)), 2) AS avg_price
    FROM home_sales
    WHERE price >= '350000' 
    GROUP BY view
    ORDER BY avg_price 
""")

# Show the result
result.show()

# Print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  33|393973.32|
|  37|394964.36|
|   6|395655.38|
|  42| 396964.5|
|   2|397389.25|
|  29|397771.65|
|  30| 397862.0|
|  38|398418.37|
|  47| 398447.5|
|  17|398474.49|
|  14|398570.03|
|   8|398592.71|
|   3| 398867.6|
|  13|398917.98|
|  19|398953.17|
|  39|399234.22|
|  18|399332.91|
|  32|399367.72|
|  20|399522.81|
|  27|399537.66|
+----+---------+
only showing top 20 rows

--- 0.20922017097473145 seconds ---


In [80]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [81]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [82]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Start the timer
start_time = time.time()

# Query on the cached data
result_cached = spark.sql("""
    SELECT view, ROUND(AVG(CAST(price AS DOUBLE)), 2) AS avg_price
    FROM home_sales
    WHERE price >= '350000'
    GROUP BY view
    ORDER BY avg_price
""")

# Show the result
result_cached.show()

# Print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|  33|393973.32|
|  37|394964.36|
|   6|395655.38|
|  42| 396964.5|
|   2|397389.25|
|  29|397771.65|
|  30| 397862.0|
|  38|398418.37|
|  47| 398447.5|
|  17|398474.49|
|  14|398570.03|
|   8|398592.71|
|   3| 398867.6|
|  13|398917.98|
|  19|398953.17|
|  39|399234.22|
|  18|399332.91|
|  32|399367.72|
|  20|399522.81|
|  27|399537.66|
+----+---------+
only showing top 20 rows

--- 0.25557994842529297 seconds ---


In [83]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
# Assuming home_sales_df is your DataFrame
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("date_built_partitioned")

In [84]:
# 11. Read the formatted parquet data.
home_sales_df=spark.read.parquet('date_built_partitioned')

In [85]:
# 12. Create a temporary table for the parquet data.
home_sales_df.createOrReplaceTempView("home_sales_parquet")

In [86]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

# Start the timer
start_time = time.time()

# Query on the cached data
result_parquet = spark.sql("""
    SELECT view, ROUND(AVG(CAST(price AS DOUBLE)), 2) AS avg_price
    FROM home_sales_parquet
    WHERE price >= '350000'
    GROUP BY view
    ORDER BY avg_price
""")

# Show the result
result_parquet.show()

# Print the runtime
print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  33|393973.32|
|  37|394964.36|
|   6|395655.38|
|  42| 396964.5|
|   2|397389.25|
|  29|397771.65|
|  30| 397862.0|
|  38|398418.37|
|  47| 398447.5|
|  17|398474.49|
|  14|398570.03|
|   8|398592.71|
|   3| 398867.6|
|  13|398917.98|
|  19|398953.17|
|  39|399234.22|
|  18|399332.91|
|  32|399367.72|
|  20|399522.81|
|  27|399537.66|
+----+---------+
only showing top 20 rows

--- 0.5662899017333984 seconds ---


In [87]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [88]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False

23/12/02 21:50:19 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar